# SIGN LANGUAGE DETECTION

**Importing Libraries**

In [2]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense , Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
from datetime import datetime


**Preparing Data for Training**

In [3]:
def load_and_preprocess_data(directory, img_size=(64, 64)):
    datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)
    
    train_data = datagen.flow_from_directory(
        "asl_alphabet_train/",
        target_size=img_size,
        color_mode='grayscale',
        batch_size=32,
        class_mode='categorical',
        subset='training'
    )
    
    val_data = datagen.flow_from_directory(
        "asl_alphabet_test/",
        target_size=img_size,
        color_mode='grayscale',
        batch_size=32,
        class_mode='categorical',
        subset='validation'
    )
    
    return train_data, val_data

train_data, val_data = load_and_preprocess_data('dataset')

Found 69600 images belonging to 1 classes.
Found 5 images belonging to 1 classes.


**Defining and Training Model**

In [4]:
def build_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

input_shape = (64, 64, 1)  # Grayscale image
num_classes = len(train_data.class_indices)
model = build_model(input_shape, num_classes)

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

model.fit(train_data, epochs=10, validation_data=val_data, callbacks=[early_stopping])
model.save('sign_language_model.h5')


Epoch 1/10
2175/2175 [==============================] - 172s 79ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
2175/2175 [==============================] - 160s 74ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
2175/2175 [==============================] - 174s 80ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
2175/2175 [==============================] - 176s 81ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000


**Implementing Real-Time Detection**

In [12]:
def preprocess_frame(frame, img_size=(64, 64)):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized_frame = cv2.resize(gray_frame, img_size)
    normalized_frame = resized_frame / 255.0
    return np.expand_dims(np.expand_dims(normalized_frame, axis=0), axis=-1)

def real_time_detection(model):
    cap = cv2.VideoCapture(0)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        processed_frame = preprocess_frame(frame)
        prediction = model.predict(processed_frame)
        sign = np.argmax(prediction)
        
        cv2.putText(frame, f"Sign: {sign}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.imshow('Sign Language Detection', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()


In [5]:
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 128)        0